<a href="https://colab.research.google.com/github/FilatovArtem/MTS_ML-Course/blob/main/HW_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8C%D1%8F_%D0%B8_%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Деревья

Рассмотрим вопрос как на самом деле происходит сплит по непрерывной фиче в дереве

Не будем останавливаться на всех возможных реализациях обучения деревьев

*   ID3
*   CART (с модификациями используется в sklearn)
*   C4.5 (J48)
*   C5.0
*   CN2
*   CHAID
*   Symmetric/Oblivious Trees (catboost)

Рассмотрим только вопрос сплита




Давайте напишем пару вспомогательных функций

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from operator import itemgetter

In [ ]:
VERY_BIG = 1e90

In [ ]:
def split_criterion(x, x_treshold, target, type = 'entropy'):
  assert len(x) == len(target)
  if type not in ['MSE', 'entropy', 'gini_impurity']:
    return 'incorrect criterion type'
  x = pd.Series(x)
  target = pd.Series(target, index = x.index)
  target_left, target_right = target[x < x_treshold], target[x >= x_treshold]
  if (not len(target_left)) or (not len(target_left)):
    return VERY_BIG
  if type == 'MSE':
    x_left, x_right = x[x < x_treshold], x[x >= x_treshold]
    prediction_left, prediction_right = np.mean(x_left), np.mean(x_right)
    mse_left, mse_right = sum((target_left - prediction_left) ** 2), sum((target_right - prediction_right) ** 2)
    return (len(target_left) * mse_left + len(target_right) * mse_right) / len(target)

  p_left, p_right = np.mean(target_left), np.mean(target_right)
  if type == 'gini_impurity':
    return 1 - (p_left ** 2 + p_right ** 2)
  if type == 'entropy':
    if (not p_left) or (not p_right):
      return VERY_BIG
    else:
      return -p_left * np.log2(p_left) -p_right * np.log2(p_right)

Давайте посмотрим на выбор сплита перебором порога в задаче классификации

In [ ]:
feature = [5, 6, 7, 8, 9, 10, 2]
target = [1, 0, 1, 0, 1, 1, 1]
t = []
for feature_treshold in feature[0:-1]:
  t.append([feature_treshold, split_criterion(feature, feature_treshold, target, type = 'entropy')])
t = sorted(t, reverse = True, key = itemgetter(1))
print(tabulate(t, headers = ['Порог', 'Значение критерия']))

  Порог    Значение критерия
-------  -------------------
      7             0.701253
      8             0.701253
      6             0.442179
      9             0.442179
      5             0.389975
     10             0.389975


Давайте убедимся что пороги 5 и 10 одинаково хорошо разбивают дерево, посчитаем для этого частоту единичек в поддеревьях

In [ ]:
feature = pd.Series(feature)
target = pd.Series(target, index = feature.index)
print(np.mean(target[feature < 5]), np.mean(target[feature >= 5]))
print(np.mean(target[feature < 10]), np.mean(target[feature >= 10]))

1.0 0.6666666666666666
0.6666666666666666 1.0


Теперь перейдем непосредственно к стратегиям сплитов

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
random_state = 88
x, y = make_classification(5_000, 10, random_state = random_state)

Теперь реализуем самую простую стратегию выбора первого узла дерева

In [ ]:
%%time
best_splits = []
for idx_feature, feature in enumerate(x.T):
  t = []
  for feature_treshold in feature[0:-1]:
    t.append([feature_treshold, split_criterion(feature, feature_treshold, y, type = 'entropy')])
  best_split_index = np.nanargmin(np.array(t)[:, 1], axis = 0)
  best_splits.append([idx_feature, t[best_split_index][0], t[best_split_index][1]])
best_splits = sorted(best_splits, reverse = True, key = itemgetter(1))
print(tabulate(best_splits, headers = ['Feature', 'Порог', 'Значение критерия']))
print('the winner is')
best_of_best_split = np.nanargmin(np.array(best_splits)[:,2], axis = 0)
print(tabulate([best_splits[best_of_best_split]], headers = ['Feature', 'Порог', 'Значение критерия']))

  Feature     Порог    Значение критерия
---------  --------  -------------------
        1   4.34112             0.499956
        9   4.18103             0.499956
        6   3.51631             0.499956
        2   3.46727             0.499956
        5   3.24306             0.964164
        4   2.29289             0.499956
        7  -1.10045             0.468756
        8  -3.53499             0.499956
        3  -3.54405             0.499956
        0  -3.85811             0.499956
the winner is
  Feature     Порог    Значение критерия
---------  --------  -------------------
        7  -1.10045             0.468756
CPU times: user 38.8 s, sys: 92.7 ms, total: 38.9 s
Wall time: 44.9 s


Долго считалось!!

Мы перебрали все фичи и все пороги -> всего около 5 000 сплитов

Как подсчитать радикально быстрее?

Через гистограммы, но это будет другое дерево, точность упадет

Подсказка: используйте функцию pd.qcut

In [ ]:
%%time
best_splits = []
for idx_feature, feature in enumerate(x.T):
  t = []
  _, bins = pd.qcut(feature, q=128, retbins=True, duplicates='drop')
  for feature_treshold in bins:
    criterion_val = split_criterion(feature, feature_treshold, y, type='entropy')
    t.append([feature_treshold, criterion_val])
  best_split_index = np.nanargmin(np.array(t)[:, 1], axis=0)
  best_splits.append([idx_feature, t[best_split_index][0], t[best_split_index][1]])
best_splits = sorted(best_splits, reverse = True, key = itemgetter(1))
print(tabulate(best_splits, headers = ['Feature', 'Порог', 'Значение критерия']))
print('the winner is')
best_of_best_split = np.nanargmin(np.array(best_splits)[:,2], axis = 0)
print(tabulate([best_splits[best_of_best_split]], headers = ['Feature', 'Порог', 'Значение критерия']))

  Feature     Порог    Значение критерия
---------  --------  -------------------
        1   4.34112             0.499956
        9   4.18103             0.499956
        6   3.51631             0.499956
        2   3.46727             0.499956
        4   2.29289             0.499956
        5   2.08983             0.98471
        3   1.76306             0.974604
        8  -1.01648             0.990269
        7  -1.10756             0.470641
        0  -2.3691              0.524808
the winner is
  Feature     Порог    Значение критерия
---------  --------  -------------------
        7  -1.10756             0.470641
CPU times: user 926 ms, sys: 3.07 ms, total: 929 ms
Wall time: 950 ms


**Задание**: при числе бинов в гистограмме равной 128 какая фича окажется в корне дерева?



1.   feature_0
2.   feature_1
3.   feature_2
4.   feature_3
5.   feature_4
6.   feature_5
7.   feature_6
8.   feature_7 <--
9.   feature_8
10.  feature_9

Стратегии выбора порогов очень разные, например в catboost:

https://catboost.ai/docs/en/concepts/quantization

# Градиентный бустинг



Мы же с вами здесь реализуем базовый кейс -- градиентный бустинг в задаче регресии


In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

In [ ]:
class simple_gradient_boosting_regressor():
  def __init__(self, n_estimators = 20, max_depth = 5, learning_rate = 0.2, random_state = 55):
    self.learning_rate = learning_rate
    self.max_depth = max_depth
    self.random_state = random_state
    self.n_estimators = n_estimators
    self.trees = []

  def fit(self, x, y, verbose = True):
    preds = np.mean(y)
    for idx in range(self.n_estimators):
      current_tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                random_state=self.random_state
            )
      residuals = y - preds
      current_tree.fit(x, residuals)
      self.trees.append(current_tree)
      preds += self.learning_rate * current_tree.predict(x)
      if verbose:
        mse = mean_squared_error(y, preds)
        print(f'MSE after {idx} tree is {mse:.3f}')

  def predict(self, x):
    preds = np.zeros(x.shape[0])
    for tree in self.trees:
      preds += self.learning_rate * tree.predict(x)
    return preds

In [ ]:
random_state = 55
x, y = make_regression(1000, 5, random_state = random_state)

In [ ]:
gbr = simple_gradient_boosting_regressor()
gbr.fit(x, y)

MSE after 0 tree is 11217.737
MSE after 1 tree is 7910.404
MSE after 2 tree is 5650.280
MSE after 3 tree is 4118.589
MSE after 4 tree is 3032.511
MSE after 5 tree is 2233.586
MSE after 6 tree is 1676.377
MSE after 7 tree is 1283.969
MSE after 8 tree is 984.848
MSE after 9 tree is 774.297
MSE after 10 tree is 603.629
MSE after 11 tree is 483.340
MSE after 12 tree is 392.615
MSE after 13 tree is 321.182
MSE after 14 tree is 267.361
MSE after 15 tree is 226.771
MSE after 16 tree is 193.463
MSE after 17 tree is 169.164
MSE after 18 tree is 149.315
MSE after 19 tree is 132.892


**Задание:**
Во сколько раз упала ошибка MSE на трейне при параметрах, указанных в коде?



1.   Не упала
2.   Упала менее чем в 2 раза
3.   Упала от 2х до 10 раз
4.   Упала от 10 до 100 раз <--
5.   Упала от ста до тысячи раз
6.   Упала более чем в тысячу раз



Для активно интересующихся и скучающих пара статей на русском

https://habr.com/ru/articles/799725/

https://habr.com/ru/companies/vk/articles/438562/